In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
from sklearn.model_selection import train_test_split

**Данный ноутбук содержит следующие части:**

*   label encoding для целевой переменной
*   генерация дополнительных фичей
*   перевод фичи с текстом в кодированную матрицу с помощью tf-df
*   эксперименты с моделью Random Forest:
*      подбор фичей на части датасета
*      подбор оптимальных гиперпараметров на части датасета


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Загрузим наш  датасет (200 обьектов ) как экперементальный и  для теста ( 50 обьектов). Так же в изначальный датасет были добавлены дополнительные признаки, такие как:
*   Year (год)
*   Month
*   Day
*   Day of week
*   season

Таргет переменную (topic)была закодирована label encording





In [3]:
# df = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/clear_data.csv')
# df.drop('Unnamed: 0', axis=1, inplace=True)

X_train = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/train_200k_texts.csv')
##Y_train = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/ytrain_200k_texts.csv')
X_test = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/test_50k_texts.csv')
#Y_test = pd.read_csv('/content/drive/Othercomputers/My Laptop/Project_1_HSE/данные/ytest_50k_texts.csv')

In [4]:
X_train

,Unnamed: 0,date,url,topic,tags,text_str,topic_le,title_lemmas,year,month,day,date_enc,seanos,day_of_week,dummy_weekday
0,337202,2010-07-12,https://lenta.ru/news/2010/07/12/explosion/,Россия,Все,запад москва квартира этажный дом происходить ...,20,дом запад москва происходить взрыв газ,2010,7,12,3966,3,0,1
1,460023,2013-03-29,https://lenta.ru/news/2013/03/29/samogon/,Бывший СССР,NaN,украина предлагать вводить уголовный ответстве...,3,украинский самогонщик пригрозить уголовный отв...,2013,3,29,4958,2,4,1
2,340676,2010-08-09,https://lenta.ru/news/2010/08/09/aliev/,Интернет и СМИ,Все,улица москва близкий время появляться социальн...,7,андрей тихонов сергей игнашевич рекламировать ...,2010,8,9,3993,3,0,1
3,88483,2004-02-28,https://lenta.ru/news/2004/02/28/chita/,Россия,Все,ход расследование взрыв читинский кафе онон за...,20,чита задерживать предполагать виновник взрыв кафе,2004,2,28,1642,1,5,0
4,939735,2021-06-18,https://lenta.ru/news/2021/06/18/iron/,Ценности,Явления,эксперт раскрывать альтернативный способ погла...,25,эксперт раскрывать способ погладить одежда утюг,2021,6,18,7957,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207551,669264,2017-06-29,https://lenta.ru/news/2017/06/29/mudcrab/,Из жизни,Звери,австралиец гривз beau greaves поймать огромный...,6,австралиец голый рука поймать огромный мангров...,2017,6,29,6508,3,3,1
207552,1021507,2022-01-14,https://lenta.ru/news/2022/01/14/perehod/,Бывший СССР,Средняя Азия,полиция алма-ата заниматься расследование пере...,3,алма-ата заниматься переходить сторона протест...,2022,1,14,8168,1,4,1
207553,840724,2020-05-31,https://lenta.ru/news/2020/05/31/vghy/,Мир,Общество,французский ученый называть возможный срок поя...,13,называть реальный срок проникновение коронавир...,2020,5,31,7575,2,6,0
207554,398616,2011-11-26,https://lenta.ru/news/2011/11/26/nato/,Мир,Все,власть пакистан ноябрь закрывать дорога которы...,13,пакистан перекрывать снабжение сила нато афган...,2011,11,26,4465,4,5,0


In [5]:
X_train['topic'].value_counts()

Россия               43390
Мир                  35240
Экономика            21982
Бывший СССР          16878
Спорт                16428
Наука и техника      13610
Культура             12918
Интернет и СМИ       11664
Из жизни              8066
Силовые структуры     7792
Путешествия           4096
Ценности              4018
Дом                   3984
Среда обитания        2809
Бизнес                1337
Моя страна            1103
Нацпроекты             851
69-я параллель         604
Забота о себе          571
Крым                   120
Культпросвет            61
Легпром                 21
Библиотека              11
Оружие                   2
Name: topic, dtype: int64

In [6]:
X_test['topic'].value_counts()

Россия               8678
Мир                  7048
Экономика            4396
Бывший СССР          3376
Спорт                3286
Наука и техника      2722
Культура             2584
Интернет и СМИ       2333
Из жизни             1613
Силовые структуры    1558
Путешествия           819
Ценности              804
Дом                   797
Среда обитания        562
Бизнес                268
Моя страна            221
Нацпроекты            170
69-я параллель        121
Забота о себе         114
Крым                   24
Культпросвет           12
Легпром                 4
Библиотека              2
Name: topic, dtype: int64

In [7]:
X_train.shape

(207556, 15)

In [8]:
X_test.shape

(41512, 15)

так как два топика ('Оружие', 'Библиотека') редко встречаются и наверняка их модель не предскажет - удалим их.

In [9]:
for topic in ['Оружие', 'Библиотека']:
  X_train = X_train.drop(X_train[X_train['topic'] == topic].index)
  X_test = X_test.drop(X_test[X_test['topic'] == topic].index)

X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)

In [10]:
X_train

,index,Unnamed: 0,date,url,topic,tags,text_str,topic_le,title_lemmas,year,month,day,date_enc,seanos,day_of_week,dummy_weekday
0,0,337202,2010-07-12,https://lenta.ru/news/2010/07/12/explosion/,Россия,Все,запад москва квартира этажный дом происходить ...,20,дом запад москва происходить взрыв газ,2010,7,12,3966,3,0,1
1,1,460023,2013-03-29,https://lenta.ru/news/2013/03/29/samogon/,Бывший СССР,NaN,украина предлагать вводить уголовный ответстве...,3,украинский самогонщик пригрозить уголовный отв...,2013,3,29,4958,2,4,1
2,2,340676,2010-08-09,https://lenta.ru/news/2010/08/09/aliev/,Интернет и СМИ,Все,улица москва близкий время появляться социальн...,7,андрей тихонов сергей игнашевич рекламировать ...,2010,8,9,3993,3,0,1
3,3,88483,2004-02-28,https://lenta.ru/news/2004/02/28/chita/,Россия,Все,ход расследование взрыв читинский кафе онон за...,20,чита задерживать предполагать виновник взрыв кафе,2004,2,28,1642,1,5,0
4,4,939735,2021-06-18,https://lenta.ru/news/2021/06/18/iron/,Ценности,Явления,эксперт раскрывать альтернативный способ погла...,25,эксперт раскрывать способ погладить одежда утюг,2021,6,18,7957,3,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207538,207551,669264,2017-06-29,https://lenta.ru/news/2017/06/29/mudcrab/,Из жизни,Звери,австралиец гривз beau greaves поймать огромный...,6,австралиец голый рука поймать огромный мангров...,2017,6,29,6508,3,3,1
207539,207552,1021507,2022-01-14,https://lenta.ru/news/2022/01/14/perehod/,Бывший СССР,Средняя Азия,полиция алма-ата заниматься расследование пере...,3,алма-ата заниматься переходить сторона протест...,2022,1,14,8168,1,4,1
207540,207553,840724,2020-05-31,https://lenta.ru/news/2020/05/31/vghy/,Мир,Общество,французский ученый называть возможный срок поя...,13,называть реальный срок проникновение коронавир...,2020,5,31,7575,2,6,0
207541,207554,398616,2011-11-26,https://lenta.ru/news/2011/11/26/nato/,Мир,Все,власть пакистан ноябрь закрывать дорога которы...,13,пакистан перекрывать снабжение сила нато афган...,2011,11,26,4465,4,5,0


In [11]:
Y_train = X_train['topic_le']
Y_test = X_test['topic_le']
Y_train.value_counts()

20    43390
13    35240
27    21982
3     16878
23    16428
16    13610
10    12918
7     11664
6      8066
21     7792
19     4096
25     4018
4      3984
24     2809
2      1337
15     1103
17      851
0       604
5       571
8       120
9        61
11       21
Name: topic_le, dtype: int64

In [12]:
date_cols = ['year', 'day_of_week', 'seanos']

Создадим новый датафрейм для числовых признаков и затем закодируем их OHE с переводом в спарс матрицу

In [13]:
X_train_num = X_train[date_cols]
X_test_num = X_test[date_cols]
X_train_num

,year,day_of_week,seanos
0,2010,0,3
1,2013,4,2
2,2010,0,3
3,2004,5,1
4,2021,4,3
...,...,...,...
207538,2017,3,3
207539,2022,4,1
207540,2020,6,2
207541,2011,5,4


In [14]:
enc = OneHotEncoder(drop='first', handle_unknown='error', sparse=True)

enc.fit(X_train_num)

train_num = enc.transform(X_train_num)
#feature_names = enc.get_feature_names(columns_cat)
test_num = enc.transform(X_test_num)

In [15]:
train_num

<207543x32 sparse matrix of type '<class 'numpy.float64'>'
	with 539621 stored elements in Compressed Sparse Row format>

**Text data transformation** 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(ngram_range=(1,1), min_df = 0.0001, max_df = 0.8)
X_train_bow = vec.fit_transform(X_train['text_str'])

In [17]:
X_test_bow = vec.transform(X_test['text_str'])

In [18]:
X_train_bow

<207543x34198 sparse matrix of type '<class 'numpy.float64'>'
	with 20105031 stored elements in Compressed Sparse Row format>

Соеденим обе матрицы  - с числовыми признаками и с закодированными текстовыми.

In [19]:
full_X_train = scipy.sparse.hstack([X_train_bow, train_num]).tocsr()
full_X_test = scipy.sparse.hstack([X_test_bow, test_num]).tocsr()

In [23]:
full_X_train

<207543x9191 sparse matrix of type '<class 'numpy.float64'>'
	with 19027643 stored elements in Compressed Sparse Row format>

Далее хотелось бы сделать подбор гипер параметров для модели случайного леса. Но в силу нехватки ресурсов для расчета - не получается . тк колаб вылетает. Проще подбирать руками гиперпараметры. Но оставлю этот код здесь на всякий случай.

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# parameter_grid = {
#             'max_depth': [30,100,200],
#             'n_estimators': [500,1000],
#             'min_samples_split':[5,20]
#         }
# model = RandomForestClassifier(random_state=123, n_jobs=-1, verbose=2)
# grid_searcher = GridSearchCV(model, parameter_grid, cv=0, verbose=2)
# grid_searcher.fit(full_X_train, Y_train)
# model_best = grid_searcher.best_estimator_
 
# print('Best params = ', model_best.get_params())

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

best_model = None
best_score = 0
best_params = ()

#for max_depth in [30, 100, 200]:
for max_depth in [100,200]:
  for n_estimators in [500]:
    for min_samples_split in [5]:
      print(f'====== start training for max_depth: {max_depth}, n_estimators: {n_estimators}, min_samples_split: {min_samples_split}')
      model = RandomForestClassifier(
        max_depth=max_depth,
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        random_state=123, 
        n_jobs=-1, 
        verbose=1
      )
      model.fit(full_X_train, Y_train)
      pred = model.predict(full_X_test)
      score = f1_score(pred, Y_test, sample_weight=None, average="micro")
      print(f'====== finished training for max_depth: {max_depth}, n_estimators: {n_estimators}, min_samples_split: {min_samples_split}, score: {score}')
      if score > best_score:
        best_score = score
        best_model = model
        best_params = (max_depth, n_estimators, min_samples_split)

print(f'Done. Best score is {best_score}, params {best_params}')

====== start training for max_depth: 100, n_estimators: 500, min_samples_split: 5


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 55.2min


In [21]:
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из 300 деревьев глубиной 100
model = RandomForestClassifier(
  n_estimators=300,
  max_depth = 100,
  #min_samples_split = 5,
  bootstrap=True,
  max_features='sqrt',
  verbose=2,
  n_jobs=-1
)

# Обучаем на тренировочных данных
model.fit(full_X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 300
building tree 2 of 300
building tree 3 of 300
building tree 4 of 300
building tree 5 of 300
building tree 6 of 300
building tree 7 of 300
building tree 8 of 300
building tree 9 of 300
building tree 10 of 300
building tree 11 of 300
building tree 12 of 300
building tree 13 of 300
building tree 14 of 300
building tree 15 of 300
building tree 16 of 300
building tree 17 of 300
building tree 18 of 300
building tree 19 of 300
building tree 20 of 300
building tree 21 of 300
building tree 22 of 300
building tree 23 of 300
building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300
building tree 28 of 300
building tree 29 of 300
building tree 30 of 300
building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300
building tree 35 of 300
building tree 36 of 300
building tree 37 of 300
building tree 38 of 300


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  9.4min


building tree 39 of 300
building tree 40 of 300
building tree 41 of 300
building tree 42 of 300
building tree 43 of 300
building tree 44 of 300
building tree 45 of 300
building tree 46 of 300
building tree 47 of 300
building tree 48 of 300
building tree 49 of 300
building tree 50 of 300
building tree 51 of 300
building tree 52 of 300
building tree 53 of 300
building tree 54 of 300
building tree 55 of 300
building tree 56 of 300
building tree 57 of 300
building tree 58 of 300
building tree 59 of 300
building tree 60 of 300
building tree 61 of 300
building tree 62 of 300
building tree 63 of 300
building tree 64 of 300
building tree 65 of 300
building tree 66 of 300
building tree 67 of 300
building tree 68 of 300
building tree 69 of 300
building tree 70 of 300
building tree 71 of 300
building tree 72 of 300
building tree 73 of 300
building tree 74 of 300
building tree 75 of 300
building tree 76 of 300
building tree 77 of 300
building tree 78 of 300
building tree 79 of 300
building tree 80

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 37.6min


building tree 160 of 300
building tree 161 of 300
building tree 162 of 300
building tree 163 of 300
building tree 164 of 300
building tree 165 of 300
building tree 166 of 300
building tree 167 of 300
building tree 168 of 300
building tree 169 of 300
building tree 170 of 300
building tree 171 of 300
building tree 172 of 300
building tree 173 of 300
building tree 174 of 300
building tree 175 of 300
building tree 176 of 300
building tree 177 of 300
building tree 178 of 300
building tree 179 of 300
building tree 180 of 300
building tree 181 of 300
building tree 182 of 300
building tree 183 of 300
building tree 184 of 300
building tree 185 of 300
building tree 186 of 300
building tree 187 of 300
building tree 188 of 300
building tree 189 of 300
building tree 190 of 300
building tree 191 of 300
building tree 192 of 300
building tree 193 of 300
building tree 194 of 300
building tree 195 of 300
building tree 196 of 300
building tree 197 of 300
building tree 198 of 300
building tree 199 of 300


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 71.1min finished


RandomForestClassifier(max_depth=100, max_features='sqrt', n_estimators=300,
                       n_jobs=-1, verbose=2)

На тесте скор 0.74

In [22]:
  from sklearn.metrics import f1_score
  pred = model.predict(full_X_test)
  score = f1_score(pred, Y_test, sample_weight=None, average="micro")
  score

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.4s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    5.6s
[Parallel(n_jobs=2)]: Done 300 out of 300 | elapsed:   11.2s finished


0.7400626355095159

In [ ]:
import pickle
with open('./model2.pkl', 'wb') as fp:
  pickle.dump(model, fp)

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из 400 деревьев и глубиной 60
model = RandomForestClassifier(
  n_estimators=400,
  max_depth = 60,
  #min_samples_split = 5,
  bootstrap=True,
  max_features='sqrt',
  verbose=2,
  n_jobs=-1
)

# Обучаем на тренировочных данных
model.fit(full_X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 400building tree 2 of 400

building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.3min


building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400
building tree 44 of 400
building tree 45 of 400
building tree 46 of 400
building tree 47 of 400
building tree 48 of 400
building tree 49 of 400
building tree 50 of 400
building tree 51 of 400
building tree 52 of 400
building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 70 of 400
building tree 71 of 400
building tree 72 of 400
building tree 73 of 400
building tree 74 of 400
building tree 75 of 400
building tree 76 of 400
building tree 77 of 400
building tree 78 of 400
building tree 79 of 400
building tree 80

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 32.7min


building tree 160 of 400
building tree 161 of 400
building tree 162 of 400
building tree 163 of 400
building tree 164 of 400
building tree 165 of 400
building tree 166 of 400
building tree 167 of 400
building tree 168 of 400
building tree 169 of 400
building tree 170 of 400
building tree 171 of 400
building tree 172 of 400
building tree 173 of 400
building tree 174 of 400
building tree 175 of 400
building tree 176 of 400
building tree 177 of 400
building tree 178 of 400
building tree 179 of 400
building tree 180 of 400
building tree 181 of 400
building tree 182 of 400
building tree 183 of 400
building tree 184 of 400
building tree 185 of 400
building tree 186 of 400
building tree 187 of 400
building tree 188 of 400
building tree 189 of 400
building tree 190 of 400
building tree 191 of 400
building tree 192 of 400
building tree 193 of 400
building tree 194 of 400
building tree 195 of 400
building tree 196 of 400
building tree 197 of 400
building tree 198 of 400
building tree 199 of 400


[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed: 74.1min


building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 81.6min finished


RandomForestClassifier(max_depth=60, max_features='sqrt', n_estimators=400,
                       n_jobs=-1, verbose=2)

На трейне скор 0.96. Модель скорее всего переобучилась

In [27]:
from sklearn.metrics import f1_score
pred_train = model.predict(full_X_train)
score_train = f1_score(pred_train, Y_train, sample_weight=None, average="micro")
score_train

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    4.1s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:   17.3s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:   39.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:   43.3s finished


0.9695677522248402

На тесте скор - 0.73 и есть редкие классы которые предсказать не может

In [23]:

pred = model.predict(full_X_test)
score = f1_score(pred, Y_test, sample_weight=None, average="micro")
score

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    1.3s
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed:    4.3s
[Parallel(n_jobs=2)]: Done 361 tasks      | elapsed:    9.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:   10.0s finished


0.7323295591423753

In [30]:
dict_topic = dict(zip(X_train.topic, X_train.topic_le))
keys = list(dict_topic.keys())
keys.sort()
dict_topic_new = {}
for key in keys:
  dict_topic_new[key] = dict_topic[key]
dict_topic_new

{'69-я параллель': 0,
 'Бизнес': 2,
 'Бывший СССР': 3,
 'Дом': 4,
 'Забота о себе': 5,
 'Из жизни': 6,
 'Интернет и СМИ': 7,
 'Крым': 8,
 'Культпросвет ': 9,
 'Культура': 10,
 'Легпром': 11,
 'Мир': 13,
 'Моя страна': 15,
 'Наука и техника': 16,
 'Нацпроекты': 17,
 'Путешествия': 19,
 'Россия': 20,
 'Силовые структуры': 21,
 'Спорт': 23,
 'Среда обитания': 24,
 'Ценности': 25,
 'Экономика': 27}

In [35]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print (metrics.classification_report (Y_train, pred_train,target_names = dict_topic))

                   precision    recall  f1-score   support

           Россия       1.00      0.68      0.81       604
      Бывший СССР       1.00      0.96      0.98      1337
   Интернет и СМИ       1.00      0.97      0.98     16878
         Ценности       1.00      0.93      0.96      3984
         Культура       1.00      0.54      0.71       571
  Наука и техника       1.00      0.87      0.93      8066
   Среда обитания       1.00      0.95      0.97     11664
            Спорт       1.00      0.78      0.87       120
         Из жизни       1.00      0.80      0.89        61
              Мир       1.00      0.97      0.99     12918
Силовые структуры       1.00      0.86      0.92        21
        Экономика       0.97      0.98      0.97     35240
       Моя страна       1.00      0.76      0.86      1103
              Дом       1.00      0.97      0.99     13610
           Бизнес       1.00      0.98      0.99       851
       Нацпроекты       1.00      0.94      0.97      4

In [37]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print (metrics.classification_report(Y_test, pred,target_names = dict_topic))

                   precision    recall  f1-score   support

           Россия       0.00      0.00      0.00       121
      Бывший СССР       0.00      0.00      0.00       268
   Интернет и СМИ       0.77      0.71      0.74      3376
         Ценности       0.89      0.53      0.66       797
         Культура       1.00      0.01      0.02       114
  Наука и техника       0.83      0.37      0.51      1613
   Среда обитания       0.78      0.40      0.53      2333
            Спорт       0.00      0.00      0.00        24
         Из жизни       0.00      0.00      0.00        12
              Мир       0.81      0.81      0.81      2584
Силовые структуры       0.00      0.00      0.00         4
        Экономика       0.70      0.83      0.76      7048
       Моя страна       1.00      0.00      0.01       221
              Дом       0.81      0.77      0.79      2722
           Бизнес       0.99      0.79      0.88       170
       Нацпроекты       0.89      0.55      0.68       

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pickle
with open('./model3.pkl', 'wb') as fp:
  pickle.dump(model, fp)

Попробуем те же гиперпараметры но уменьшим порог в матрице tf-df. Те слов станет больше - что возможно поможет предсказать редкие классы.

In [27]:
from sklearn.ensemble import RandomForestClassifier

# Создаём модель леса из 400 деревьев
model = RandomForestClassifier(
  n_estimators=400,
  max_depth = 60,
  #min_samples_split = 5,
  bootstrap=True,
  max_features='sqrt',
  random_state=123,
  verbose=2,
  n_jobs=-1
)

# Обучаем на тренировочных данных
model.fit(full_X_train, Y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400
building tree 34 of 400
building tree 35 of 400
building tree 36 of 400


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.5min


building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400
building tree 44 of 400
building tree 45 of 400
building tree 46 of 400
building tree 47 of 400
building tree 48 of 400
building tree 49 of 400
building tree 50 of 400
building tree 51 of 400
building tree 52 of 400
building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 70 of 400
building tree 71 of 400
building tree 72 of 400
building tree 73 of 400
building tree 74 of 400
building tree 75 of 400
building tree 76 of 400
building tree 77 of 400
building tree 78

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 11.1min


building tree 158 of 400
building tree 159 of 400
building tree 160 of 400
building tree 161 of 400
building tree 162 of 400
building tree 163 of 400
building tree 164 of 400
building tree 165 of 400
building tree 166 of 400
building tree 167 of 400
building tree 168 of 400
building tree 169 of 400
building tree 170 of 400
building tree 171 of 400
building tree 172 of 400
building tree 173 of 400
building tree 174 of 400
building tree 175 of 400
building tree 176 of 400
building tree 177 of 400
building tree 178 of 400
building tree 179 of 400
building tree 180 of 400
building tree 181 of 400
building tree 182 of 400
building tree 183 of 400
building tree 184 of 400
building tree 185 of 400
building tree 186 of 400
building tree 187 of 400
building tree 188 of 400
building tree 189 of 400
building tree 190 of 400
building tree 191 of 400
building tree 192 of 400
building tree 193 of 400
building tree 194 of 400
building tree 195 of 400
building tree 196 of 400
building tree 197 of 400


[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 25.3min


building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400
building tree 398 of 400
building tree 399 of 400
building tree 400 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 28.3min finished


RandomForestClassifier(max_depth=60, max_features='sqrt', n_estimators=400,
                       n_jobs=-1, random_state=123, verbose=2)

In [21]:
dict_topic = dict(zip(X_train.topic, X_train.topic_le))
keys = list(dict_topic.keys())
keys.sort()
dict_topic_new = {}
for key in keys:
  dict_topic_new[key] = dict_topic[key]
dict_topic_new

{'69-я параллель': 0,
 'Бизнес': 2,
 'Бывший СССР': 3,
 'Дом': 4,
 'Забота о себе': 5,
 'Из жизни': 6,
 'Интернет и СМИ': 7,
 'Крым': 8,
 'Культпросвет ': 9,
 'Культура': 10,
 'Легпром': 11,
 'Мир': 13,
 'Моя страна': 15,
 'Наука и техника': 16,
 'Нацпроекты': 17,
 'Путешествия': 19,
 'Россия': 20,
 'Силовые структуры': 21,
 'Спорт': 23,
 'Среда обитания': 24,
 'Ценности': 25,
 'Экономика': 27}

In [28]:
from sklearn.metrics import f1_score
pred_train = model.predict(full_X_train)
score_train = f1_score(pred_train, Y_train, sample_weight=None, average="micro")
score_train

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:   12.7s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:   28.1s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:   31.4s finished


0.9577436964869931

In [29]:
pred = model.predict(full_X_test)
score = f1_score(pred, Y_test, sample_weight=None, average="micro")
score

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed:    6.3s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    6.9s finished


0.7224765116839316

In [25]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

print (metrics.classification_report (Y_train, pred_train,target_names = dict_topic_new))

                   precision    recall  f1-score   support

   69-я параллель       1.00      0.27      0.42       604
           Бизнес       1.00      0.65      0.79      1337
      Бывший СССР       0.99      0.91      0.95     16878
              Дом       1.00      0.81      0.89      3984
    Забота о себе       1.00      0.11      0.20       571
         Из жизни       1.00      0.63      0.77      8066
   Интернет и СМИ       0.99      0.79      0.88     11664
             Крым       1.00      0.35      0.52       120
    Культпросвет        1.00      0.30      0.46        61
         Культура       0.97      0.93      0.95     12918
          Легпром       1.00      0.62      0.76        21
              Мир       0.90      0.96      0.93     35240
       Моя страна       1.00      0.40      0.57      1103
  Наука и техника       0.98      0.93      0.95     13610
       Нацпроекты       1.00      0.93      0.96       851
      Путешествия       1.00      0.80      0.89      4

In [26]:
print (metrics.classification_report(Y_test, pred,target_names = dict_topic_new))

                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00       121
           Бизнес       0.00      0.00      0.00       268
      Бывший СССР       0.79      0.62      0.69      3376
              Дом       0.86      0.48      0.62       797
    Забота о себе       0.00      0.00      0.00       114
         Из жизни       0.89      0.26      0.40      1613
   Интернет и СМИ       0.81      0.33      0.47      2333
             Крым       0.00      0.00      0.00        24
    Культпросвет        0.00      0.00      0.00        12
         Культура       0.81      0.79      0.80      2584
          Легпром       0.00      0.00      0.00         4
              Мир       0.68      0.83      0.75      7048
       Моя страна       0.00      0.00      0.00       221
  Наука и техника       0.82      0.75      0.78      2722
       Нацпроекты       1.00      0.65      0.79       170
      Путешествия       0.94      0.44      0.60       

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Итоговый скор на тестовых данных - 0.74

Итоги


*   Random forest - модель,требующая больших вычистлительных ресурсов.
*   Чтобы поднять скор - нужно работать над гиперпараметрами, но это не позволет сделать количество памяти. а расчеты на GPU random forest не поддерживает.
     Каждая модель считалась от 30 мин  - 1.5 часов. 
